In [31]:
import os

In [32]:
%pwd

'/home/muhammed-shafeeh/AI_ML/DL_kidney_tumor_End_to_End_project'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/home/muhammed-shafeeh/AI_ML/DL_kidney_tumor_End_to_End_project'

In [34]:
import dagshub
dagshub.init(repo_owner='shafeeh011', repo_name='DL_kidney_tumor_End_to_End_project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "shafeeh011/DL_kidney_tumor_End_to_End_project"

Repository shafeeh011/DL_kidney_tumor_End_to_End_project initialized!

🏃 View run unruly-skink-760 at: https://dagshub.com/shafeeh011/DL_kidney_tumor_End_to_End_project.mlflow/#/experiments/0/runs/c7cf14d94ae542bba86f4f7a036a1cfc
🧪 View experiment at: https://dagshub.com/shafeeh011/DL_kidney_tumor_End_to_End_project.mlflow/#/experiments/0


In [35]:
import tensorflow as tf

In [36]:
model = tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [43]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [44]:
from src.dlproject.constants import *
from src.dlproject.utils.common import read_yaml,create_directories,save_json

In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_evalution_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/trained_model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/shafeeh011/DL_kidney_tumor_End_to_End_project.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [46]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [47]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.path_of_model
        )
        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        score = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("artifacts/score.json"), data=score)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [48]:
try:
    config = ConfigurationManager()
    val_config = config.get_evalution_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 1.0000 - loss: 0.1739


2025/02/11 19:53:07 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/11 19:53:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/11 19:53:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


🏃 View run shivering-lynx-923 at: https://dagshub.com/shafeeh011/DL_kidney_tumor_End_to_End_project.mlflow/#/experiments/0/runs/f70f795d60644a538d95aeb71a641967
🧪 View experiment at: https://dagshub.com/shafeeh011/DL_kidney_tumor_End_to_End_project.mlflow/#/experiments/0
